# Predicción de Calidad de Café

**Objetivo**  
Predecir la calidad del café basados en el Puntaje de Taza.

---

### 1. Análisis de datos y selección de variables de entrada (X)
- Cargar y explorar el conjunto de datos.  
- Definir las variables predictoras (**X**), cuidando que no se filtre información del target.

### 2. Preprocesamiento
- **X** (features):
  - Imputación de valores faltantes  
  - Normalización / estandarización  
  - Codificación de variables categóricas  
- **Y** (Puntaje de Taza):
  - Revisión de distribución  
  - Transformaciones si aplica (e.g., escalado, discretización)

### 3. Entrenamiento de modelos de regresión
- Modelo 1: _Linear Regression_ (u otro algoritmo lineal)  
- Modelo 2: _Random Forest Regressor_ (u otro algoritmo no lineal)  

### 4. Explicabilidad
- Aplicar técnicas de interpretabilidad:
  - LIME  
  - SHAP  
  - Importancia de características (Feature Importance)

### 5. Reporte de resultados
- Métricas de evaluación: RMSE, MAE, R²  
- Comparación de desempeño entre modelos  
- Visualizaciones e interpretaciones  


In [1]:
#Primero importamos todas las librerias que vamos a utilizar
import os 
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Importamos las librerias de sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


### 1.1 Análisis de datos
- Cargar y explorar el conjunto de datos.  

In [52]:
#Primero Cargamos los datos almacenados en los archivos .xlsx
#Las primeras 5 filas de los archivos son encabezados, por lo cual las saltamos

dict_Calidad = pd.read_excel('CC_FT_17_Calidad.xlsx', 
                             sheet_name=['CONTROL CALIDAD CAFE TRILLADO J','Sheet2'],
                             skiprows=5)

#Accedemos a las hojas específicas del diccionario
#Quitamos las columnas que no son necesarias para el análisis, por ejemplo verificación física del café tostado tiene 
#Unicamente un valor y no aporta información relevante para el análisis

df_Calidad1 = dict_Calidad['CONTROL CALIDAD CAFE TRILLADO J']

#Las columnas sin nombre en este caso solo tienen un valor: 'C', por lo cual procedemos a eliminarlas.
df_Calidad1 = df_Calidad1.loc[:, ~df_Calidad1.columns.str.startswith('Unnamed')]

#quita espacios al principio y al final de cada nombre
df_Calidad1.columns = df_Calidad1.columns.str.strip()

df_Calidad1 = df_Calidad1.drop(columns=['FECHA','VERIFICACIÓN FISICA CAFÉ TOSTADO', 'LIBERACIÓN DE LOTE', 'RESPONSABLE'])

#Por ultimo eliminamos las filas 0 y 1 que no aportan información relevante y las ultimas filas.

df_Calidad1 = df_Calidad1.drop(index=[0,1])
df_Calidad1 = df_Calidad1.iloc[:-17]


# Realizamos una visualización rápida de los datos en cada hoja

display(df_Calidad1.head())
display(df_Calidad1.tail())


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-190722,Madre Laura,765.00,10.9,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,09-190722,Tabi Natural,204.00,10.2,14,"Frutas maduras, nibs de cacao, acidez brillant...",85
4,10-190722,Don Mario,165.00,10.7,14,"Panela, durazno, miel, acidez brillante citric...",84.5
5,07-19-07-22,Don Felix,0.45,10.5,14,"Moras maduras, chocolate negro, acidez media c...",84.5
6,01-291022,Madre Laura,105.00,10.7,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
73,22-170624,El Ocaso - Caturron,48.0,10.3,15,"Chocolate dulce, nuez moscada, acidez jugosa, ...",87
74,01-030724,Madre Laura,250.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
75,23-150724,Esteban Robledo,60.0,10.6,14,"Panela, almendras, acidez citrica jugosa como ...",85
76,02-150724,Madre Laura Natural,175.0,10.8,14,"Cacao, frutos rojos, cuerpo cremoso, acidez br...",85
77,17-270724,Familia Bedoya Castaño,35.0,10.8,14,"Melao de panela, arandanos, cuerpo cremoso, ac...",87


In [53]:
#Ahora hacemos un tratamiento similar con la segunda hoja del archivo Excel
df_Calidad2 = dict_Calidad['Sheet2']

df_Calidad2 = df_Calidad2.loc[:, ~df_Calidad2.columns.str.startswith('Unnamed')]
df_Calidad2.columns = df_Calidad2.columns.str.strip()
df_Calidad2 = df_Calidad2.drop(columns=['FECHA','VERIFICACIÓN FISICA CAFÉ TOSTADO', 'LIBERACIÓN DE LOTE', 'RESPONSABLE'])
df_Calidad2 = df_Calidad2.drop(index=[0,1])
df_Calidad2 = df_Calidad2.iloc[:-12]

display(df_Calidad2.head())
display(df_Calidad2.tail())

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
2,01-300822,Madre Laura,32.0,"10,5",14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
3,01-131022,Madre Laura,79.8,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
4,01-181022,Madre Laura,38.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
5,05-181022,Doña Dolly,43.0,10,14,"Chocolate dulce,fresas,miel,cuerpo cremoso,aci...",85
6,01-291022,Madre Laura,272.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
29,01-271023,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
30,01-100124,Madre Laura,20.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
31,01-020424,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
32,01-200624,Madre Laura,14.0,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
33,01-180724,Madre Laura,20.0,10.2,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


In [54]:
#Ahora unimos los dos dataframes en uno solo por filas

df_Calidad = pd.concat([df_Calidad1, df_Calidad2], axis=0, ignore_index=True)

display(df_Calidad.head())
display(df_Calidad.tail())

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
0,01-190722,Madre Laura,765.00,10.9,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
1,09-190722,Tabi Natural,204.00,10.2,14,"Frutas maduras, nibs de cacao, acidez brillant...",85
2,10-190722,Don Mario,165.00,10.7,14,"Panela, durazno, miel, acidez brillante citric...",84.5
3,07-19-07-22,Don Felix,0.45,10.5,14,"Moras maduras, chocolate negro, acidez media c...",84.5
4,01-291022,Madre Laura,105.00,10.7,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
103,01-271023,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
104,01-100124,Madre Laura,20.0,10.4,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
105,01-020424,Madre Laura,20.0,10.5,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
106,01-200624,Madre Laura,14.0,10.8,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84
107,01-180724,Madre Laura,20.0,10.2,14,"Chocolate negro, toque frutal, cuerpo medio, a...",84


Las notas de cata del café constituyen variables categóricas muy relevantes para el modelo. Para incorporarlas sin introducir un orden arbitrario, aplicamos one-hot encoding: creamos una columna por cada nota de cata (por ejemplo, “Sabor afrutado”, “Cuerpo ligero”, “Aroma floral”, etc.) y asignamos un 1 si el café presenta dicha nota, o un 0 en caso contrario.

Este enfoque:

- Mantiene intacta la información cualitativa de cada nota sin reducirla a un único valor numérico.
- Facilita que los algoritmos de regresión interpreten correctamente la presencia o ausencia de cada característica sensorial.
- Permite evaluar la importancia individual de cada nota mediante técnicas de explicabilidad (Feature Importance, SHAP, LIME).

De este modo, garantizamos que el modelo reciba entradas limpias y libres de sesgos de orden, y podamos extraer conclusiones precisas sobre qué atributos de cata influyen más en la calidad final del café.


In [55]:
#Primero limpiamos los datos de la columna 'NOTAS DE CATACIÓN' para convertirlos en una lista de notas

# Limpia espacios redundantes y divide en lista
df_Calidad['NOTAS DE CATACIÓN'] = (
    df_Calidad['NOTAS DE CATACIÓN']
      .fillna('')                           # en caso de NaN
      .str.replace(r'\s*,\s*', ',', regex=True)
      .str.split(',')                       # ahora cada fila es lista de notas
)

display(df_Calidad.head())

,LOTE,DENOMINACIÓN/ MARCA,CANTIDAD,%H,MALLAS,NOTAS DE CATACIÓN,PUNTAJE
0,01-190722,Madre Laura,765.00,10.9,14,"[Chocolate negro, toque frutal, cuerpo medio, ...",84
1,09-190722,Tabi Natural,204.00,10.2,14,"[Frutas maduras, nibs de cacao, acidez brillan...",85
2,10-190722,Don Mario,165.00,10.7,14,"[Panela, durazno, miel, acidez brillante citri...",84.5
3,07-19-07-22,Don Felix,0.45,10.5,14,"[Moras maduras, chocolate negro, acidez media ...",84.5
4,01-291022,Madre Laura,105.00,10.7,14,"[Chocolate negro, toque frutal, cuerpo medio, ...",84
